##### Quiz) 위키피디아에서 제공되는 지진 데이터를 활용한 Q&A 챗봇을 만드시오.

##### 조건)
1. 거대 언어 모델(llm) : gpt-4o-mini
1. 임베딩 모델 : BAAI/bge-m3
1. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
1. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
1. 벡터DB : Chroma
1. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
1. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
1. 출력 파서 : StrOutputParser

##### 질문)
1. 지진파의 종류에 대해서 알려줘
1. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
1. 지진을 주제로 한 영화는 어떤 게 있어?

##### 답변) 아래 내용이 포함되면 정답으로 간주
1. P파, S파, 표면파
1. 칠레 발디비아 지진
1. 대지진, 일본 침몰, 2012

In [8]:
# 1. 거대 언어 모델(llm) : gpt-4o-mini
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
# 2. 임베딩 모델 : BAAI/bge-m3
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "BAAI/bge-m3"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [10]:
# 3. 문서 : 위키피디아에서 제공하는 지진 내용 전체 (WikipediaLoader 사용, 한국어, doc_content_chars_max 40000)
from langchain_community.document_loaders import WikipediaLoader
loader = WikipediaLoader(query="지진", lang="ko", load_max_docs=1, doc_content_chars_max=40000)

In [11]:
# 4. 텍스트 분리(splitter) : CharacterTextSplitter 사용 (chunk_size=1000, chunk_overlap=100)
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

docs = loader.load_and_split(text_splitter=text_splitter)

Created a chunk of size 1268, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1451, which is longer than the specified 1000
Created a chunk of size 2325, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1213, which is longer than the specified 1000
Created a chunk of size 1260, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of size 1998, which is longer than the specified 1000
Created a chunk of size 1141, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000


In [12]:
# 5. 벡터DB : Chroma
from langchain_chroma import Chroma
db = Chroma.from_documents(docs, hf_embeddings)

In [13]:
# 6. 검색기(retriever) : 기본값으로 하되 최대 검색 청크 수 k=3 으로 설정
retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
# 7. 프롬프트 템플릿 : langchain-ai/retrieval-qa-chat 활용
from langchain import hub
prompt_template = hub.pull("langchain-ai/retrieval-qa-chat")

c:\Python311\Lib\site-packages\langsmith\client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [15]:
# 8. 출력 파서 : StrOutputParser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [16]:
# Chain
from langchain_core.runnables import RunnablePassthrough

rag_chain = {
    "context": retriever,
    "input": RunnablePassthrough()
} | prompt_template | llm | parser

In [ ]:
# 결과 확인

# 질문)
# 1. 지진파의 종류에 대해서 알려줘
# 2. 지금까지 있었던 지진 중 가장 규모가 큰 것은?
# 3. 지진을 주제로 한 영화는 어떤 게 있어?

# 답변)
# 1. P파, S파, 표면파
# 2. 칠레 발디비아 지진
# 3. 대지진, 일본 침몰, 2012

In [17]:
rag_chain.invoke("지진파의 종류에 대해서 알려줘")

'지진파의 종류에는 다음과 같은 세 가지가 있습니다:\n\n1. **P파 (종방향파 혹은 압축파)**: 이 지진파는 물질의 입자가 이동하는 방향과 동일한 방향으로 압축과 신장을 반복하면서 전파됩니다. P파는 지진이 발생했을 때 가장 먼저 도달하는 지진파로, 일반적으로 다른 파동보다 빠르게 움직입니다.\n\n2. **S파 (횡방향파 혹은 전단파)**: S파는 물질의 입자가 이동하는 방향과 수직으로 진동하는 방식으로 전파됩니다. S파는 P파보다 느리게 도달하며, 지진 발생 후 2번째로 오는 파동입니다.\n\n3. **표면파**: 이들은 지구 표면을 따라 전파되는 지진파로, 레일리파와 러브파가 포함됩니다. 표면파는 P파와 S파보다 느리지만, 대부분의 지진 피해를 주는 주된 원인입니다.\n\n각 지진파는 서로 다른 속도와 특성을 가지고 있으며, 이를 통해 지진의 진원, 깊이, 그리고 지구 내부 구조를 분석할 수 있습니다.'

In [18]:
rag_chain.invoke("지금까지 있었던 지진 중 가장 규모가 큰 것은?")

'지금까지 있었던 지진 중 가장 규모가 큰 것은 1960년 5월 22일 발생한 칠레 발디비아 지진으로, 그 규모는 M9.5입니다.'

In [19]:
rag_chain.invoke("지진을 주제로 한 영화는 어떤 게 있어?")

'지진을 주제로 한 영화에는 다음과 같은 작품들이 있습니다:\n\n1. 《대지진》(1974년) - 1971년 샌페르난도 지진에서 영감을 받아 만들어졌습니다.\n2. 《일본 침몰》(1973년) - 고마쓰 사쿄의 소설을 바탕으로 한 영화입니다.\n3. 《2012》(2009년) - 대규모 자연재해를 다룬 영화로, 여러 재난을 포함하고 있습니다.\n\n이 외에도 지진을 다룬 다양한 영화가 존재합니다.'